# Import Libraries and Input Image

In [2]:
import cv2
import numpy as np
import os

# Make output directory
os.makedirs("images", exist_ok=True)


# Load image
img = cv2.imread('input.jpg')

# 1) Reduce Intensity Level

In [ ]:
def reduce_intensity_levels(img, levels):
    factor = 256 // levels
    reduced_img = (img // factor) * factor
    return reduced_img

for lvl in [2, 4, 8, 16]:
    reduced = reduce_intensity_levels(img, lvl)
    cv2.imwrite(f'images/reduced_{lvl}_levels.jpg', reduced)

# 2) Spatial Averaging (Smoothing)

In [ ]:
def average_filter(img, ksize):
    return cv2.blur(img, (ksize, ksize))

for k in [3, 10, 20]:
    avg = average_filter(img, k)
    cv2.imwrite(f'images/avg_{k}x{k}.jpg', avg)

# 3) Image Rotation

In [ ]:
def rotate_image(img, angle):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Compute new bounding dimensions
    cos = np.abs(matrix[0, 0])
    sin = np.abs(matrix[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    
    # Adjust rotation matrix to account for translation
    matrix[0, 2] += (nW / 2) - center[0]
    matrix[1, 2] += (nH / 2) - center[1]
    
    rotated = cv2.warpAffine(img, matrix, (nW, nH))
    return rotated

rot_45 = rotate_image(img, 45)
rot_90 = rotate_image(img, 90)

cv2.imwrite('images/rotated_45.jpg', rot_45)
cv2.imwrite('images/rotated_90.jpg', rot_90)

True

# 4) Reduce Resolution via Block Averaging

In [ ]:
def block_average(img, block_size):
    h, w, c = img.shape
    reduced_img = np.copy(img)
    for y in range(0, h, block_size):
        for x in range(0, w, block_size):
            y_end = min(y + block_size, h)
            x_end = min(x + block_size, w)
            block = img[y:y_end, x:x_end]
            avg_val = block.reshape(-1, c).mean(axis=0).astype(np.uint8)
            reduced_img[y:y_end, x:x_end] = avg_val
    return reduced_img

for b in [3, 5, 7]:
    block = block_average(img, b)
    cv2.imwrite(f'images/block_{b}x{b}.jpg', block)